In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.optimizers import Adam

from classification import preprocessing, custom_vgg16

# Test scores

In [ ]:
# Import test set
test_dir = 'data/test_set/'
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    seed=123,
    image_size=(180, 180),
    batch_size=32,
    label_mode="categorical")

In [ ]:
# Preprocess test set
test_ds = preprocessing(test_ds)

In [ ]:
# Evaluate all models on the train set

test_scores = pd.DataFrame(index=['accuracy', 'loss'])

for aug in ['no_aug', 'flip', 'mosaic', 'tapestry', 'starry_night', 'starry_night_flip', 'starry_night_tapestry']:
    weights_path = f'checkpoints/{aug}/second_training.ckpt'

    model = custom_vgg16(180, 180, 101)
    model.load_weights(weights_path)
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    loss, acc = model.evaluate(test_ds, verbose=1)
    test_scores[aug] = [acc, loss]

In [ ]:
# Save the test results
test_scores.to_csv('classification_results/test_scores.csv', index=True)

# Training and validation error

In [ ]:
# Create a dataframe with the final train and validation accuracies and losses for each model
train_val_scores = pd.DataFrame(index=['Train accuracy', 'Train loss', 'Validation accuracy', 'Validation loss'])

for aug in ['no_aug', 'flip', 'mosaic', 'tapestry', 'starry_night', 'starry_night_flip', 'starry_night_tapestry']:
    history = pd.read_csv(f'histories/model_history_{aug}.csv')

    best_epoch = history['val_accuracy'].idxmax()
    train_val_scores.loc[['Train accuracy', 'Train loss', 'Validation accuracy', 'Validation loss'], aug] = history.loc[best_epoch].drop('epoch').to_numpy()


In [ ]:
# Import test losses and accuracies (computed in the previous part)
test_scores = pd.read_csv('classification_results/test_scores.csv', index_col=0)

In [ ]:
# Merge train, validation and test results
scores = (pd.concat([train_val_scores, test_scores])
          .rename(index={'accuracy': 'Test', 'loss': 'Test loss', 'Train accuracy': 'Train', 'Validation accuracy': 'Validation'},
                  columns={'no_aug': 'No augmentation', 'flip': 'Flip', 'tapestry': 'Tapestry', 'mosaic': 'Mosaic', 'starry_night': 'Starry night', 'starry_night_flip': 'Starry night + Flip', 'starry_night_tapestry': 'Starry night + Tapestry'})
          .loc[['Train', 'Validation', 'Test']]
          )

In [ ]:
# Plot accuracies

fig, ax = plt.subplots()
scores.plot.bar(cmap='Set2', ax=ax)

ax.set_ylim(0.9, 1)
ax.tick_params('x', labelrotation=0)
ax.set_ylabel('Accuracy')
ax.grid(axis='y', linestyle='--', which='major', color='gray', linewidth=0.5, alpha=0.7)
ax.set_yticks(np.arange(0.9, 1.01, 0.01))

plt.savefig('plots/accuracies.png')

plt.show()